In [2]:
%autosave 1
import os
import argparse
import numpy as np
import matplotlib.pyplot as plt
import random
from matplotlib.patches import Rectangle
from z3 import *

Autosaving every 1 seconds


In [3]:
instance_name_int = 40
instance_name_str = str(instance_name_int) + 'x' + str(instance_name_int) + '.txt'
base_dir = 'Instances/'
with open(base_dir + instance_name_str) as f:
    content = f.readlines()
content = [x.strip() for x in content]
# remove empty strings
content = [x for x in content if x]
width = int(content[0].split(' ')[0])
height = int(content[0].split(' ')[1])
blobs_count = int(content[1])
blobs = []
for i in range(2, len(content)):
    blobs.append([int(content[i].split(' ')[0]), int(content[i].split(' ')[1])])

In [4]:
# variables: array composed by pairs of (x, y) coordinates which are the starting point of the 
# respsective blobs in blobs array. This is a variable so it is not itialized yet, 
# is what we have to find.
xy_coord = [ [ Int("o_{}_{}".format(i+1, j+1)) for j in range(2) ] for i in range(blobs_count) ]

In [5]:
# constraint for ensure that the domain of the origins should be [0, width[ and [0, height[
in_domain = [ And(xy_coord[i][0] >= 0, xy_coord[i][0] < width, xy_coord[i][1] >= 0, xy_coord[i][1] < height)
             for i in range(blobs_count)] 

In [6]:
# each piece must fit in the paper roll
fit_paper = [ And(blobs[i][0] + xy_coord[i][0] <= width, blobs[i][1] + xy_coord[i][1] <= height) 
            for i in range(blobs_count)] 

In [7]:
# non-overlap constraint using global constrint
over = []
for i in range(blobs_count):
    for j in range(blobs_count):
        if (i<j):
            over.append(Or(xy_coord[i][0] + blobs[i][0]  <= xy_coord[j][0], xy_coord[j][0] + blobs[j][0] <= xy_coord[i][0], xy_coord[i][1] + blobs[i][1] <= xy_coord[j][1], xy_coord[j][1] + blobs[j][1] <= xy_coord[i][1]))

In [8]:
# implied cumulative constraints
imp = []
for i in range(width):
    for j in range(blobs_count):
        implied.append(Sum([If(And(xy_coord[j][0] <= i, i < xy_coord[j][0] + blobs[j][0]), blobs[j][1],0) for j in range(blobs_count)]) <= height)

for i in range(height):
    for j in range(blobs_count):
        implied.append(Sum([If(And(xy_coord[j][1] <= i, i < xy_coord[j][1] + blobs[j][1]), blobs[j][0],0) for j in range(blobs_count)]) <= width)

### Simmetry breaking

In [9]:
#this constraint forces the first (biggest area object) to be placed into thefirst, bottom-left quadrant of the box
simmetry_b = [ And(xy_coord[0][0] <= (width/2 - blobs[0][0]/2), xy_coord[0][1] <= (height/2 - blobs[0][1]/2))] 

In [10]:
# TODO: syntax error
# identical rectangle constraint
#identical = [If(And(blobs[i][0] == blobs[i+1][0], blobs[i][1] == blobs[i+1][1]), 
#   Or(O[i][0] < O[i+1][0], And(O[i][0] == O[i+1][0], O[i][1] < O[i+1][1])),
#   False) for i in range(blobs_count)]

In [11]:
constraints = domain + fit_paper + overe + simmetry_b

In [12]:
'''
Solver() creates a general purpose solver.
Constraints can be added using the method add.
We say the constraints have been asserted in the solver. 
The method check() solves the asserted constraints. 
The result is sat (satisfiable) if a solution was found. 
The result is unsat (unsatisfiable) if no solution exists.
We may also say the system of asserted constraints is infeasible. 
Finally, a solver may fail to solve a system of constraints and unknown is returned.
'''
s = Solver()
s.add(constraints)

In [14]:
if s.check() == sat:
    m = s.model()

    print("{} {}".format(height, width))
    print("{}".format(blobs_count))

    for i in range(blobs_count):
        print("{:<1} {:<3} {:<1} {:<2}".format(blobs[i][0], blobs[i][1], str(m[xy_coord[i][0]]), str(m[xy_coord[i][1]])))
    print("\n{}\n".format(s.statistics()))
else: 
    print("Failed to solve")

40 40
20
3 3   0 0 
3 4   37 36
3 5   37 0 
3 6   0 34
3 7   37 5 
3 8   37 12
3 9   0 3 
3 10  0 12
3 12  0 22
3 16  37 20
3 40  3 0 
4 4   6 14
4 14  6 0 
4 22  6 18
5 3   32 37
5 4   32 25
5 8   32 29
5 12  32 0 
5 13  32 12
22 40  10 0 

(:added-eqs                      94
 :arith-bound-propagations-cheap 184543
 :arith-bound-propagations-lp    38498
 :arith-cheap-eqs                94
 :arith-conflicts                9724
 :arith-lower                    126297
 :arith-make-feasible            76056
 :arith-max-columns              848
 :arith-max-rows                 760
 :arith-propagations             184543
 :arith-upper                    105132
 :binary-propagations            31035
 :conflicts                      11305
 :decisions                      146159
 :del-clause                     1146
 :final-checks                   2
 :max-memory                     15.75
 :memory                         7.88
 :minimized-lits                 5789
 :mk-bool-var                 